In [ ]:
import os
import json
import torch
import random
import numpy as np
import importlib
import utils
importlib.reload(utils)
from utils import *
from tqdm.notebook import tqdm
from collections import defaultdict
from transformer_lens import HookedTransformer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using device: {DEVICE}")

using device: cuda


In [2]:
pythia = HookedTransformer.from_pretrained(
    model_name='EleutherAI/pythia-70m-deduped',
    device=DEVICE,
    local_files_only=True
)
pythia.eval()

with open('./corpus/pythia_cluster_tokens.json', 'r') as f:
    pythia_high_act_tokens = json.load(f)
with open('./corpus/pythia_token_corpus.json', 'r') as f:
    pythia_token_corpus = json.load(f)
with open('./corpus/pythia_at_token.json', 'r') as f:
    pythia_at_token = json.load(f)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [2]:
def run_with_raw_model(model, text):
    input_ids = model.to_tokens(text, prepend_bos=False)

    with torch.no_grad():
        last_token_logits = model(input_ids)[0, -1, :]

    probs = torch.softmax(last_token_logits, dim=-1)

    # get top10 tokens and their probs
    top10_probs, top10_indices = torch.topk(probs, 10)

    return top10_indices, top10_probs

def run_with_hooked_model(model, text, hook_pos, direction, scale):
    def scale_direction(dir, scale):
        def fn_hook(act, hook):
            act[:, :, :] += scale * dir
            return act
        return fn_hook

    # normalize the direction vector
    norm_direction = direction / torch.norm(direction)
    hook_direction = norm_direction
    input_ids = model.to_tokens(text, prepend_bos=False)
    with torch.no_grad():
        last_token_logits = model.run_with_hooks(
            input_ids,
            fwd_hooks=[
                (hook_pos, scale_direction(hook_direction, scale))
            ]
        )[0, -1, :]

    probs = torch.softmax(last_token_logits, dim=-1)

    # get top10 tokens and their probs
    top10_probs, top10_tokens = torch.topk(probs, 10)

    return top10_tokens, top10_probs

def get_random_unit_vector(dim, device=DEVICE):
    random_vec = torch.randn(dim, device=device)
    return random_vec / torch.norm(random_vec)

def nested_dict():
    return defaultdict(dict)

def nested_defaultdict():
    return defaultdict(nested_dict)

def nested_defaultdict_3():
    return defaultdict(nested_defaultdict)

def convert_tensors_to_python_types(obj):
    if isinstance(obj, torch.Tensor):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_tensors_to_python_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_tensors_to_python_types(item) for item in obj]
    else:
        return obj

In [ ]:
scale_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 17, 20]
scales = [-x for x in scale_values[::-1]] + scale_values
TEST_CLUSTERS = 5
TEST_FEATURES = 5

pythia_token_embed = pythia.embed.W_E.detach().to(DEVICE)
pythia_test_result = defaultdict(nested_defaultdict_3)

for layer_type, layer_type_info in tqdm(pythia_high_act_tokens.items()):
    for layer_index, layer_index_info in tqdm(layer_type_info.items()):
        # select clusters randoly
        cluster_ids = list(layer_index_info.keys())
        random.shuffle(cluster_ids)

        # get basic intervention info
        sae_dec = get_sae_weights_from_local('pythia', layer_type, layer_index)
        hook_pos = get_hook_position(f'{layer_index}-{layer_type}')
        sae_features = get_sae_features_by_layer('pythia', layer_type, layer_index)
        exp_embeddings = get_feature_explanation_embeddings_by_layer('pythia', layer_type, layer_index)
        
        # generate random intervention vectors for baseline
        vec_dim = sae_dec.shape[1]
        random_sae_vec = get_random_unit_vector(vec_dim)
        random_grad_vec = get_random_unit_vector(vec_dim)
        
        selected_clusters = 0
        for cluster_id in cluster_ids:
            if selected_clusters >= TEST_CLUSTERS:
                break

            features = pythia_high_act_tokens[layer_type][layer_index][cluster_id]
            feature_ids = []
            semantics = []
            for feature in features:
                if feature['direction'] is not None:
                    feature_ids.append(feature['feature_id'])
                    semantics.append(feature['direction'])

            # get high-interference feature with different semantics
            outlier, sim = get_outlier(semantics)
            if outlier is None:
                continue
            
            attack_feature_id = feature_ids[outlier]
            attack_vec = sae_dec[attack_feature_id]
            selected_clusters += 1

            # get
            for feat in pythia_high_act_tokens[layer_type][layer_index][cluster_id]:
                if feat['feature_id'] == attack_feature_id:
                    attack_token = feat['token'][:10]
                    attack_token_ids = get_token_ids(pythia, attack_token)
                    attack_token_strs = [pythia.to_string(x) for x in attack_token_ids]
                    break
            
            # get gradient vector extracted from the activation texts
            high_gradient_vectors = []
            if layer_type in ['att', 'mlp']:
                feature_data = sae_features[str(attack_feature_id)]
                activation_data = feature_data['activations'][:3]
                
                for activation in activation_data:
                    if 'tokens' not in activation:
                        continue
                        
                    token_strs = activation['tokens']
                    token_position = activation['maxValueTokenIndex']
                    
                    try:
                        grad_vec = get_gradient_attack_vec(
                            model=pythia,
                            token_strs=token_strs,
                            token_position=token_position,
                            layer_idx=int(layer_index),
                            layer_type=layer_type
                        )
                        high_gradient_vectors.append(grad_vec.detach().cpu().numpy())
                    except ValueError as e:
                        print(f"Warning: {e}")
                        continue
            
            ref_vec = sae_dec[attack_feature_id]
            if isinstance(ref_vec, torch.Tensor):
                ref_vec = ref_vec.cpu()
            if isinstance(sae_dec, torch.Tensor):
                sae_dec = sae_dec.cpu()
            interference_values = cosine_similarity(ref_vec.reshape(1, -1), sae_dec).squeeze()
            
            # randomly choose a target features
            valid_features = []
            for feature in features:
                if feature['feature_id'] == attack_feature_id:
                    continue
                if str(feature['feature_id']) not in pythia_token_corpus[layer_type][layer_index][cluster_id]:
                    continue
                valid_features.append(feature)

            selected_features = random.sample(valid_features, min(TEST_FEATURES, len(valid_features)))
            for feature in selected_features:
                # get intervention vector's metadata
                attack_info = get_sae_and_gradient_attack_vectors(
                    'pythia', pythia, layer_type, layer_index, feature['feature_id'],
                    exp_embeddings, sae_features, sae_dec
                )
                
                # do experiment with random intervention vector
                attack_info['random_sae'] = {
                    'attack_feature_id': None,
                    'attack_token_set': [],
                    'attack_token_strs': [],
                    'interference': 0.0,
                    'attack_vector': random_sae_vec.cpu().numpy().tolist()
                }
                
                # prepare high-interference feature's intervention info
                feature_id_int = feature['feature_id']
                attack_info['high'].update({
                    'attack_feature_id': attack_feature_id,
                    'attack_token_set': attack_token_ids,
                    'attack_token_strs': attack_token_strs,
                    'interference': interference_values[feature_id_int],
                    'attack_vector': attack_vec.tolist()
                })
                
                if high_gradient_vectors and layer_type in ['att', 'mlp']:
                    attack_info['high']['gradient_attack_vecs'] = high_gradient_vectors

                # get testing corpus
                attacked_tk, sentences = next(
                    iter(pythia_token_corpus[layer_type][layer_index][cluster_id][str(feature['feature_id'])].items()))
                
                # get top-activating tokens of the target feature
                attacked_token_set = attack_info['self']['attacked_token_set']
                attacked_token_strs = attack_info['self']['attacked_token_strs']
                attacked_feature_high_tokens = attacked_token_set

                feature_results = {
                    'raw_next_token': attacked_tk,
                    'sentences': sentences,
                    'tests': {},
                    'attacked_feature_id': feature_id_int,
                    'attacked_highest_token': pythia.to_string(attacked_token_set[0]) if len(attacked_token_set) > 0 else ""
                }
                
                # do experiment on each sentence
                for sentence_idx, sentence in enumerate(sentences):
                    sentence_results = {}
                    
                    # get raw output for comparison
                    raw_top10_tokens, raw_top10_probs = run_with_raw_model(pythia, sentence)
                    
                    # get baseline metrics
                    raw_sim_with_attacked_tokens = get_overall_similarity(
                        pythia_token_embed, raw_top10_tokens, attacked_token_set, raw_top10_probs)
                    
                    raw_tokens_list = raw_top10_tokens.cpu().numpy().tolist()
                    baseline_kendall, baseline_spearman = get_kendall_and_spearman_metrics(
                        raw_tokens_list, attacked_feature_high_tokens)
                    baseline_overlap = get_overlap(raw_top10_tokens, raw_top10_probs, attacked_feature_high_tokens)
                    
                    sentence_results['baseline'] = {
                        'attacked_tokens_sim': raw_sim_with_attacked_tokens,
                        'kendall_tau': baseline_kendall,
                        'spearman': baseline_spearman,
                        'overlap': baseline_overlap
                    }
                    
                    # interference levels
                    attack_types = ['self', 'high', 'hmid', 'mid', 'low', 'rand', 'random_sae']
                    
                    # steering with feature directions
                    for attack_type in attack_types:
                        if attack_info[attack_type] is None:
                            continue
                        
                        attack_vector = torch.tensor(attack_info[attack_type]['attack_vector'], device=DEVICE)
                        current_attack_token_set = attack_info[attack_type]['attack_token_set']
                        
                        attack_feature_id_to_save = (
                            attack_feature_id if attack_type == 'high' 
                            else attack_info[attack_type].get('feature_id', None)
                        )
                        attack_highest_token = ""
                        if len(current_attack_token_set) > 0:
                            attack_highest_token = pythia.to_string(current_attack_token_set[0])
                        
                        max_attack_sim = 0
                        max_attacked_sim = 0
                        max_abs_kendall = 0
                        max_abs_spearman = 0
                        max_overlap = 0
                        
                        best_attack_scale = 0
                        best_attacked_scale = 0
                        best_kendall_scale = 0
                        best_kendall_value = 0
                        best_spearman_scale = 0
                        best_spearman_value = 0
                        best_overlap_scale = 0
                        
                        for scale in scales:
                            hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                pythia, sentence, hook_pos, attack_vector, scale)
                            
                            current_attacked_sim = get_overall_similarity(
                                pythia_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                            
                            current_attack_sim = get_overall_similarity(
                                pythia_token_embed, hooked_top10_tokens, current_attack_token_set, hooked_top10_probs)
                            
                            hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                            current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                hooked_tokens_list, attacked_feature_high_tokens)
                            
                            current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                            
                            if current_attacked_sim > max_attacked_sim:
                                max_attacked_sim = current_attacked_sim
                                best_attacked_scale = scale
                            
                            if current_attack_sim > max_attack_sim:
                                max_attack_sim = current_attack_sim
                                best_attack_scale = scale
                            
                            if abs(current_kendall) > max_abs_kendall:
                                max_abs_kendall = abs(current_kendall)
                                best_kendall_scale = scale
                                best_kendall_value = current_kendall
                            
                            if abs(current_spearman) > max_abs_spearman:
                                max_abs_spearman = abs(current_spearman)
                                best_spearman_scale = scale
                                best_spearman_value = current_spearman
                            
                            if current_overlap > max_overlap:
                                max_overlap = current_overlap
                                best_overlap_scale = scale
                        
                        # collect experiment results of feature direction intervention
                        sentence_results[attack_type] = {
                            'vector_type': 'sae',
                            'attack_feature_id': attack_feature_id_to_save,
                            'attack_highest_token': attack_highest_token,
                            'attacked_tokens_sim': max_attacked_sim,
                            'attacked_tokens_scale': best_attacked_scale,
                            'attack_tokens_sim': max_attack_sim,
                            'attack_tokens_scale': best_attack_scale,
                            'kendall_tau': best_kendall_value,
                            'kendall_tau_scale': best_kendall_scale,
                            'spearman': best_spearman_value,
                            'spearman_scale': best_spearman_scale,
                            'overlap': max_overlap,
                            'overlap_scale': best_overlap_scale
                        }
                    
                    # steering with gradient vector
                    if layer_type in ['att', 'mlp']:
                        for attack_type in attack_types[:-1]:
                            if attack_info[attack_type] is None or 'gradient_attack_vecs' not in attack_info[attack_type]:
                                continue
                                
                            gradient_vectors = attack_info[attack_type]['gradient_attack_vecs']
                            current_attack_token_set = attack_info[attack_type]['attack_token_set']
                            attack_feature_id_to_save = (
                                attack_feature_id if attack_type == 'high'
                                else attack_info[attack_type].get('feature_id', None)
                            )
                            attack_highest_token = ""
                            if len(current_attack_token_set) > 0:
                                attack_highest_token = pythia.to_string(current_attack_token_set[0])
                                
                            gradient_results = []
                            for vec_idx, grad_vec in enumerate(gradient_vectors):
                                grad_vec = torch.tensor(grad_vec, device=DEVICE)
                                
                                max_grad_attack_sim = 0
                                max_grad_attacked_sim = 0
                                max_grad_abs_kendall = 0
                                max_grad_abs_spearman = 0
                                max_grad_overlap = 0
                                
                                best_grad_attack_scale = 0
                                best_grad_attacked_scale = 0
                                best_grad_kendall_scale = 0
                                best_grad_kendall_value = 0
                                best_grad_spearman_scale = 0
                                best_grad_spearman_value = 0
                                best_grad_overlap_scale = 0
                                
                                for scale in scales:
                                    hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                        pythia, sentence, hook_pos, grad_vec, scale)
                                    
                                    current_attacked_sim = get_overall_similarity(
                                        pythia_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                                    
                                    current_attack_sim = get_overall_similarity(
                                        pythia_token_embed, hooked_top10_tokens, current_attack_token_set, hooked_top10_probs)
                                    
                                    hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                                    current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                        hooked_tokens_list, attacked_feature_high_tokens)
                                    
                                    current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                                    
                                    if current_attacked_sim > max_grad_attacked_sim:
                                        max_grad_attacked_sim = current_attacked_sim
                                        best_grad_attacked_scale = scale
                                    
                                    if current_attack_sim > max_grad_attack_sim:
                                        max_grad_attack_sim = current_attack_sim
                                        best_grad_attack_scale = scale
                                    
                                    if abs(current_kendall) > max_grad_abs_kendall:
                                        max_grad_abs_kendall = abs(current_kendall)
                                        best_grad_kendall_scale = scale
                                        best_grad_kendall_value = current_kendall
                                    
                                    if abs(current_spearman) > max_grad_abs_spearman:
                                        max_grad_abs_spearman = abs(current_spearman)
                                        best_grad_spearman_scale = scale
                                        best_grad_spearman_value = current_spearman
                                    
                                    if current_overlap > max_grad_overlap:
                                        max_grad_overlap = current_overlap
                                        best_grad_overlap_scale = scale
                                
                                gradient_results.append({
                                    'vec_idx': vec_idx,
                                    'attack_feature_id': attack_feature_id_to_save,
                                    'attack_highest_token': attack_highest_token,
                                    'attacked_tokens_sim': max_grad_attacked_sim,
                                    'attacked_tokens_scale': best_grad_attacked_scale,
                                    'attack_tokens_sim': max_grad_attack_sim,
                                    'attack_tokens_scale': best_grad_attack_scale,
                                    'kendall_tau': best_grad_kendall_value,
                                    'kendall_tau_scale': best_grad_kendall_scale,
                                    'spearman': best_grad_spearman_value,
                                    'spearman_scale': best_grad_spearman_scale,
                                    'overlap': max_grad_overlap,
                                    'overlap_scale': best_grad_overlap_scale
                                })
                            
                            if gradient_results:
                                sentence_results[f"{attack_type}_grad"] = gradient_results
                        
                        random_grad_results = []
                        grad_vec = random_grad_vec.to(DEVICE)
                        
                        max_grad_attack_sim = 0
                        max_grad_attacked_sim = 0
                        max_grad_abs_kendall = 0
                        max_grad_abs_spearman = 0
                        max_grad_overlap = 0
                        
                        best_grad_attack_scale = 0
                        best_grad_attacked_scale = 0
                        best_grad_kendall_scale = 0
                        best_grad_kendall_value = 0
                        best_grad_spearman_scale = 0
                        best_grad_spearman_value = 0
                        best_grad_overlap_scale = 0
                        
                        for scale in scales:
                            hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                pythia, sentence, hook_pos, grad_vec, scale)
                            
                            current_attacked_sim = get_overall_similarity(
                                pythia_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                            
                            hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                            current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                hooked_tokens_list, attacked_feature_high_tokens)
                            
                            current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                            
                            if current_attacked_sim > max_grad_attacked_sim:
                                max_grad_attacked_sim = current_attacked_sim
                                best_grad_attacked_scale = scale
                            
                            if abs(current_kendall) > max_grad_abs_kendall:
                                max_grad_abs_kendall = abs(current_kendall)
                                best_grad_kendall_scale = scale
                                best_grad_kendall_value = current_kendall
                            
                            if abs(current_spearman) > max_grad_abs_spearman:
                                max_grad_abs_spearman = abs(current_spearman)
                                best_grad_spearman_scale = scale
                                best_grad_spearman_value = current_spearman
                            
                            if current_overlap > max_grad_overlap:
                                max_grad_overlap = current_overlap
                                best_grad_overlap_scale = scale
                        
                        # record experiment results of steering with random vector
                        random_grad_results.append({
                            'vec_type': 'random',
                            'attack_feature_id': None,
                            'attack_highest_token': "",
                            'attacked_tokens_sim': max_grad_attacked_sim,
                            'attacked_tokens_scale': best_grad_attacked_scale,
                            'attack_tokens_sim': 0,
                            'attack_tokens_scale': 0,
                            'kendall_tau': best_grad_kendall_value,
                            'kendall_tau_scale': best_grad_kendall_scale,
                            'spearman': best_grad_spearman_value,
                            'spearman_scale': best_grad_spearman_scale,
                            'overlap': max_grad_overlap,
                            'overlap_scale': best_grad_overlap_scale
                        })
                        
                        sentence_results['random_grad'] = random_grad_results
                    
                    feature_results['tests'][sentence_idx] = sentence_results
                
                pythia_test_result[layer_type][layer_index][cluster_id][str(feature['feature_id'])] = feature_results
                torch.cuda.empty_cache()

# save experiment results
pythia_test_result_converted = convert_tensors_to_python_types(pythia_test_result)
with open('./pythia_test_result_v3_1.json', 'w') as f:
    json.dump(pythia_test_result_converted, f, indent=2)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

1 5
1 2
2 1
2 1
2 1
2 1
2 1
2 2
2 1
2 2
2 2
2 1
2 1
2 1
2 1
2 1
2 1
2 4
2 1
2 1
2 1
2 4
2 1
2 2
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 5
2 2
2 5
2 1
2 2
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 3
2 1
2 1
2 1
2 1
2 2
2 3
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 2
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 2
2 1
2 1
2 1
2 1
2 5
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 3
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 2
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 3
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 2
2 1
2 1
2 1
2 2
2 1
2 1
2 1
2 1
2 5
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1


  0%|          | 0/6 [00:00<?, ?it/s]

/home/walker/anaconda3/envs/interp-arena/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 4
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 5
5 1
5 1
5 1
5 1
5 5
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1


  0%|          | 0/6 [00:00<?, ?it/s]

1 3
4 1
4 1
4 1
4 5
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 5
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 5
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1


In [4]:
gpt2 = HookedTransformer.from_pretrained(
    model_name='gpt2-small',
    device=DEVICE,
    local_files_only=True
)
gpt2.eval()
print()

with open('./corpus/gpt2_cluster_tokens.json', 'r') as f:
    gpt2_high_act_tokens = json.load(f)
with open('./corpus/gpt2_token_corpus.json', 'r') as f:
    gpt2_token_corpus = json.load(f)
with open('./corpus/gpt2_at_token.json', 'r') as f:
    gpt2_at_token = json.load(f)

Loaded pretrained model gpt2-small into HookedTransformer



In [ ]:
scale_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 17, 20]
scales = [-x for x in scale_values[::-1]] + scale_values
TEST_CLUSTERS = 5
TEST_FEATURES = 5

gpt2_token_embed = gpt2.embed.W_E.detach().to(DEVICE)
gpt2_test_result = defaultdict(nested_defaultdict_3)

for layer_type, layer_type_info in tqdm(gpt2_high_act_tokens.items()):
    if layer_type not in ['att', 'mlp', 'res_mid', 'res_post']:
        continue
        
    for layer_index, layer_index_info in tqdm(layer_type_info.items()):
        cluster_ids = list(layer_index_info.keys())
        random.shuffle(cluster_ids)

        sae_dec = get_sae_weights('gpt2', layer_type, layer_index).detach().to(DEVICE)
        hook_pos = get_hook_position(f'{layer_index}-{layer_type}')
        sae_features = get_sae_features_by_layer('gpt2', layer_type, layer_index)
        exp_embeddings = get_feature_explanation_embeddings_by_layer('gpt2', layer_type, layer_index)
        
        vec_dim = sae_dec.shape[1]
        random_sae_vec = get_random_unit_vector(vec_dim)
        random_grad_vec = get_random_unit_vector(vec_dim)
        
        selected_clusters = 0
        for cluster_id in cluster_ids:
            if selected_clusters >= TEST_CLUSTERS:
                break

            features = gpt2_high_act_tokens[layer_type][layer_index][cluster_id]
            feature_ids = []
            semantics = []
            for feature in features:
                if feature['direction'] is not None:
                    feature_ids.append(feature['feature_id'])
                    semantics.append(feature['direction'])

            outlier, sim = get_outlier(semantics)
            if outlier is None:
                continue
            
            attack_feature_id = feature_ids[outlier]
            attack_vec = sae_dec[attack_feature_id]
            selected_clusters += 1

            for feat in gpt2_high_act_tokens[layer_type][layer_index][cluster_id]:
                if feat['feature_id'] == attack_feature_id:
                    attack_token = feat['token'][:10]
                    attack_token_ids = get_token_ids(gpt2, attack_token)
                    attack_token_strs = [gpt2.to_string(x) for x in attack_token_ids]
                    break
            
            high_gradient_vectors = []
            if layer_type in ['att', 'mlp']:
                feature_data = sae_features[str(attack_feature_id)]
                activation_data = feature_data['activations'][:3]
                
                for activation in activation_data:
                    if 'tokens' not in activation:
                        continue
                        
                    token_strs = activation['tokens']
                    token_position = activation['maxValueTokenIndex']
                    
                    try:
                        grad_vec = get_gradient_attack_vec(
                            model=gpt2,
                            token_strs=token_strs,
                            token_position=token_position,
                            layer_idx=int(layer_index),
                            layer_type=layer_type
                        )
                        high_gradient_vectors.append(grad_vec.detach().cpu().numpy())
                    except ValueError as e:
                        print(f"Warning: {e}")
                        continue
            
            ref_vec = sae_dec[attack_feature_id]
            if isinstance(ref_vec, torch.Tensor):
                ref_vec = ref_vec.cpu()
            if isinstance(sae_dec, torch.Tensor):
                sae_dec = sae_dec.cpu()
            interference_values = cosine_similarity(ref_vec.reshape(1, -1), sae_dec).squeeze()
            
            valid_features = []
            for feature in features:
                if feature['feature_id'] == attack_feature_id:
                    continue
                if str(feature['feature_id']) not in gpt2_token_corpus[layer_type][layer_index][cluster_id]:
                    continue
                valid_features.append(feature)

            selected_features = random.sample(valid_features, min(TEST_FEATURES, len(valid_features)))
            for feature in selected_features:
                attack_info = get_sae_and_gradient_attack_vectors(
                    'gpt2', gpt2, layer_type, layer_index, feature['feature_id'],
                    exp_embeddings, sae_features, sae_dec
                )
                
                attack_info['random_sae'] = {
                    'attack_feature_id': None,
                    'attack_token_set': [],
                    'attack_token_strs': [],
                    'interference': 0.0,
                    'attack_vector': random_sae_vec.cpu().numpy().tolist()
                }
                
                feature_id_int = feature['feature_id']
                attack_info['high'].update({
                    'attack_feature_id': attack_feature_id,
                    'attack_token_set': attack_token_ids,
                    'attack_token_strs': attack_token_strs,
                    'interference': interference_values[feature_id_int],
                    'attack_vector': attack_vec.cpu().numpy().tolist()
                })
                
                if high_gradient_vectors and layer_type in ['att', 'mlp']:
                    attack_info['high']['gradient_attack_vecs'] = high_gradient_vectors

                attacked_tk, sentences = next(
                    iter(gpt2_token_corpus[layer_type][layer_index][cluster_id][str(feature['feature_id'])].items()))
                
                attacked_token_set = attack_info['self']['attacked_token_set']
                attacked_token_strs = attack_info['self']['attacked_token_strs']
                attacked_feature_high_tokens = attacked_token_set

                feature_results = {
                    'raw_next_token': attacked_tk,
                    'sentences': sentences,
                    'tests': {},
                    'attacked_feature_id': feature_id_int,
                    'attacked_highest_token': gpt2.to_string(attacked_token_set[0]) if len(attacked_token_set) > 0 else ""
                }
                
                for sentence_idx, sentence in enumerate(sentences):
                    sentence_results = {}
                    
                    raw_top10_tokens, raw_top10_probs = run_with_raw_model(gpt2, sentence)
                    
                    raw_sim_with_attacked_tokens = get_overall_similarity(
                        gpt2_token_embed, raw_top10_tokens, attacked_token_set, raw_top10_probs)
                    
                    raw_tokens_list = raw_top10_tokens.cpu().numpy().tolist()
                    baseline_kendall, baseline_spearman = get_kendall_and_spearman_metrics(
                        raw_tokens_list, attacked_feature_high_tokens)
                    baseline_overlap = get_overlap(raw_top10_tokens, raw_top10_probs, attacked_feature_high_tokens)
                    
                    sentence_results['baseline'] = {
                        'attacked_tokens_sim': raw_sim_with_attacked_tokens,
                        'kendall_tau': baseline_kendall,
                        'spearman': baseline_spearman,
                        'overlap': baseline_overlap
                    }
                    
                    attack_types = ['self', 'high', 'hmid', 'mid', 'low', 'rand', 'random_sae']
                    
                    for attack_type in attack_types:
                        if attack_info[attack_type] is None:
                            continue
                        
                        attack_vector = torch.tensor(attack_info[attack_type]['attack_vector'], device=DEVICE)
                        current_attack_token_set = attack_info[attack_type]['attack_token_set']
                        
                        attack_feature_id_to_save = (
                            attack_feature_id if attack_type == 'high' 
                            else attack_info[attack_type].get('feature_id', None)
                        )
                        attack_highest_token = ""
                        if len(current_attack_token_set) > 0:
                            attack_highest_token = gpt2.to_string(current_attack_token_set[0])
                        
                        max_attack_sim = 0
                        max_attacked_sim = 0
                        max_abs_kendall = 0
                        max_abs_spearman = 0
                        max_overlap = 0
                        
                        best_attack_scale = 0
                        best_attacked_scale = 0
                        best_kendall_scale = 0
                        best_kendall_value = 0
                        best_spearman_scale = 0
                        best_spearman_value = 0
                        best_overlap_scale = 0
                        
                        for scale in scales:
                            hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                gpt2, sentence, hook_pos, attack_vector, scale)
                            
                            current_attacked_sim = get_overall_similarity(
                                gpt2_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                            
                            current_attack_sim = get_overall_similarity(
                                gpt2_token_embed, hooked_top10_tokens, current_attack_token_set, hooked_top10_probs)
                            
                            hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                            current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                hooked_tokens_list, attacked_feature_high_tokens)
                            
                            current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                            
                            if current_attacked_sim > max_attacked_sim:
                                max_attacked_sim = current_attacked_sim
                                best_attacked_scale = scale
                            
                            if current_attack_sim > max_attack_sim:
                                max_attack_sim = current_attack_sim
                                best_attack_scale = scale
                            
                            if abs(current_kendall) > max_abs_kendall:
                                max_abs_kendall = abs(current_kendall)
                                best_kendall_scale = scale
                                best_kendall_value = current_kendall
                            
                            if abs(current_spearman) > max_abs_spearman:
                                max_abs_spearman = abs(current_spearman)
                                best_spearman_scale = scale
                                best_spearman_value = current_spearman
                            
                            if current_overlap > max_overlap:
                                max_overlap = current_overlap
                                best_overlap_scale = scale
                        
                        sentence_results[attack_type] = {
                            'vector_type': 'sae',
                            'attack_feature_id': attack_feature_id_to_save,
                            'attack_highest_token': attack_highest_token,
                            'attacked_tokens_sim': max_attacked_sim,
                            'attacked_tokens_scale': best_attacked_scale,
                            'attack_tokens_sim': max_attack_sim,
                            'attack_tokens_scale': best_attack_scale,
                            'kendall_tau': best_kendall_value,
                            'kendall_tau_scale': best_kendall_scale,
                            'spearman': best_spearman_value,
                            'spearman_scale': best_spearman_scale,
                            'overlap': max_overlap,
                            'overlap_scale': best_overlap_scale
                        }
                    
                    if layer_type in ['att', 'mlp']:
                        for attack_type in attack_types[:-1]:
                            if attack_info[attack_type] is None or 'gradient_attack_vecs' not in attack_info[attack_type]:
                                continue
                                
                            gradient_vectors = attack_info[attack_type]['gradient_attack_vecs']
                            current_attack_token_set = attack_info[attack_type]['attack_token_set']
                            attack_feature_id_to_save = (
                                attack_feature_id if attack_type == 'high'
                                else attack_info[attack_type].get('feature_id', None)
                            )
                            attack_highest_token = ""
                            if len(current_attack_token_set) > 0:
                                attack_highest_token = gpt2.to_string(current_attack_token_set[0])
                                
                            gradient_results = []
                            for vec_idx, grad_vec in enumerate(gradient_vectors):
                                grad_vec = torch.tensor(grad_vec, device=DEVICE)
                                
                                max_grad_attack_sim = 0
                                max_grad_attacked_sim = 0
                                max_grad_abs_kendall = 0
                                max_grad_abs_spearman = 0
                                max_grad_overlap = 0
                                
                                best_grad_attack_scale = 0
                                best_grad_attacked_scale = 0
                                best_grad_kendall_scale = 0
                                best_grad_kendall_value = 0
                                best_grad_spearman_scale = 0
                                best_grad_spearman_value = 0
                                best_grad_overlap_scale = 0
                                
                                for scale in scales:
                                    hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                        gpt2, sentence, hook_pos, grad_vec, scale)
                                    
                                    current_attacked_sim = get_overall_similarity(
                                        gpt2_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                                    
                                    current_attack_sim = get_overall_similarity(
                                        gpt2_token_embed, hooked_top10_tokens, current_attack_token_set, hooked_top10_probs)
                                    
                                    hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                                    current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                        hooked_tokens_list, attacked_feature_high_tokens)
                                    
                                    current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                                    
                                    if current_attacked_sim > max_grad_attacked_sim:
                                        max_grad_attacked_sim = current_attacked_sim
                                        best_grad_attacked_scale = scale
                                    
                                    if current_attack_sim > max_grad_attack_sim:
                                        max_grad_attack_sim = current_attack_sim
                                        best_grad_attack_scale = scale
                                    
                                    if abs(current_kendall) > max_grad_abs_kendall:
                                        max_grad_abs_kendall = abs(current_kendall)
                                        best_grad_kendall_scale = scale
                                        best_grad_kendall_value = current_kendall
                                    
                                    if abs(current_spearman) > max_grad_abs_spearman:
                                        max_grad_abs_spearman = abs(current_spearman)
                                        best_grad_spearman_scale = scale
                                        best_grad_spearman_value = current_spearman
                                    
                                    if current_overlap > max_grad_overlap:
                                        max_grad_overlap = current_overlap
                                        best_grad_overlap_scale = scale
                                
                                gradient_results.append({
                                    'vec_idx': vec_idx,
                                    'attack_feature_id': attack_feature_id_to_save,
                                    'attack_highest_token': attack_highest_token,
                                    'attacked_tokens_sim': max_grad_attacked_sim,
                                    'attacked_tokens_scale': best_grad_attacked_scale,
                                    'attack_tokens_sim': max_grad_attack_sim,
                                    'attack_tokens_scale': best_grad_attack_scale,
                                    'kendall_tau': best_grad_kendall_value,
                                    'kendall_tau_scale': best_grad_kendall_scale,
                                    'spearman': best_grad_spearman_value,
                                    'spearman_scale': best_grad_spearman_scale,
                                    'overlap': max_grad_overlap,
                                    'overlap_scale': best_grad_overlap_scale
                                })
                            
                            if gradient_results:
                                sentence_results[f"{attack_type}_grad"] = gradient_results
                        
                        random_grad_results = []
                        grad_vec = random_grad_vec.to(DEVICE)
                        
                        max_grad_attack_sim = 0
                        max_grad_attacked_sim = 0
                        max_grad_abs_kendall = 0
                        max_grad_abs_spearman = 0
                        max_grad_overlap = 0
                        
                        best_grad_attack_scale = 0
                        best_grad_attacked_scale = 0
                        best_grad_kendall_scale = 0
                        best_grad_kendall_value = 0
                        best_grad_spearman_scale = 0
                        best_grad_spearman_value = 0
                        best_grad_overlap_scale = 0
                        
                        for scale in scales:
                            hooked_top10_tokens, hooked_top10_probs = run_with_hooked_model(
                                gpt2, sentence, hook_pos, grad_vec, scale)
                            
                            current_attacked_sim = get_overall_similarity(
                                gpt2_token_embed, hooked_top10_tokens, attacked_token_set, hooked_top10_probs)
                            
                            hooked_tokens_list = hooked_top10_tokens.cpu().numpy().tolist()
                            current_kendall, current_spearman = get_kendall_and_spearman_metrics(
                                hooked_tokens_list, attacked_feature_high_tokens)
                            
                            current_overlap = get_overlap(hooked_top10_tokens, hooked_top10_probs, attacked_feature_high_tokens)
                            
                            if current_attacked_sim > max_grad_attacked_sim:
                                max_grad_attacked_sim = current_attacked_sim
                                best_grad_attacked_scale = scale
                            
                            if abs(current_kendall) > max_grad_abs_kendall:
                                max_grad_abs_kendall = abs(current_kendall)
                                best_grad_kendall_scale = scale
                                best_grad_kendall_value = current_kendall
                            
                            if abs(current_spearman) > max_grad_abs_spearman:
                                max_grad_abs_spearman = abs(current_spearman)
                                best_grad_spearman_scale = scale
                                best_grad_spearman_value = current_spearman
                            
                            if current_overlap > max_grad_overlap:
                                max_grad_overlap = current_overlap
                                best_grad_overlap_scale = scale
                        
                        random_grad_results.append({
                            'vec_type': 'random',
                            'attack_feature_id': None,
                            'attack_highest_token': "",
                            'attacked_tokens_sim': max_grad_attacked_sim,
                            'attacked_tokens_scale': best_grad_attacked_scale,
                            'attack_tokens_sim': 0,
                            'attack_tokens_scale': 0,
                            'kendall_tau': best_grad_kendall_value,
                            'kendall_tau_scale': best_grad_kendall_scale,
                            'spearman': best_grad_spearman_value,
                            'spearman_scale': best_grad_spearman_scale,
                            'overlap': max_grad_overlap,
                            'overlap_scale': best_grad_overlap_scale
                        })
                        
                        sentence_results['random_grad'] = random_grad_results
                    
                    feature_results['tests'][sentence_idx] = sentence_results
                
                gpt2_test_result[layer_type][layer_index][cluster_id][str(feature['feature_id'])] = feature_results

                torch.cuda.empty_cache()

gpt2_test_result_converted = convert_tensors_to_python_types(gpt2_test_result)
with open('./gpt2_test_result_v3_1.json', 'w') as f:
    json.dump(gpt2_test_result_converted, f, indent=2)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

/home/walker/anaconda3/envs/interp-arena/lib/python3.11/site-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


1 4
1 2
1 2


  0%|          | 0/12 [00:00<?, ?it/s]

1 3
1 3
1 2
1 4


  0%|          | 0/12 [00:00<?, ?it/s]

1 2
1 6
1 3
1 2
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 4
3 5
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 2
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 3
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 2
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 6
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1
3 1


  0%|          | 0/12 [00:00<?, ?it/s]

5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 2
5 1
5 1
5 5
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 4
5 1
5 6
5 1
5 1
5 1
5 4
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 1
5 1
5 1
5 3
5 2
5 1
5 1
5 3
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 1
5 5
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 2
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 3
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 5
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 6
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
5 1
